env: scGPT

# import

In [19]:
import scanpy as sc
import pandas as pd
import numpy as np
from glob import glob
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
from torch_geometric.data import Data
from torch_geometric.data import DataLoader

import pickle
import sys
import requests

from types import MethodType
import importlib
from scperturb import *
import anndata as ad

In [20]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, precision_recall_curve
from tqdm import tqdm

# 在L1000上跑corrlation method并存储结果

In [21]:
# - get cell line name
common_cell_line = \
{   'A549': 'A549',
    'HEPG2': 'HepG2',
    'HT29': 'HT29',
    'MCF7': 'MCF7',
    # 'SKBR3': 'SK-BR-3',
    'SW480': 'SW480',
    'PC3': 'PC3',
    'A375': 'A375',
} # L1000 cell line : single-cell cell line

# - read adata_L1000, this is processed data
adata_L1000 = sc.read('/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/benchmark_data/L1000/GSE92742/adata_gene_pert.h5ad')
adata_L1000

/data1/lichen/anaconda3/envs/scGPT/lib/python3.9/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


AnnData object with n_obs × n_vars = 36720 × 978
    obs: 'sig_id', 'pert_id', 'pert_iname', 'pert_type', 'cell_id', 'pert_dose', 'pert_dose_unit', 'pert_idose', 'pert_time', 'pert_time_unit', 'pert_itime', 'distil_id'
    var: 'pr_gene_id', 'pr_gene_symbol', 'pr_gene_title', 'pr_is_lm', 'pr_is_bing'

In [27]:
from tqdm import tqdm
from sklearn.metrics import precision_recall_curve, auc

save_prefix_2 = 'mean_v2'

# mean: top gene set
# mean_v2: L1000 gene set

for cell_line_bulk in list(common_cell_line.keys())[:]:
    cell_line_single = common_cell_line[cell_line_bulk]
    print('='*20, f'cell line is {cell_line_single}')
    
    #===================prepare data
    if cell_line_bulk in ['PC3', 'A375']:
        save_dir_adata = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/benchmark_data/L1000/single_cell_data/SCP542'
        adata_rna = sc.read(os.path.join(save_dir_adata, cell_line_bulk, f'adata_{cell_line_bulk}.h5ad'))
        
        # - read adata_rna_raw
        save_dir = f'/nfs/public/lichen/data/single_cell/cell_line/SCP542/process/{cell_line_bulk}'
        adata_rna_raw = sc.read(os.path.join(save_dir, f'adata.h5ad'))
    
    else:
        save_dir_adata = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/benchmark_data/L1000/single_cell_data/CNP0003658'
        adata_rna = sc.read(os.path.join(save_dir_adata, cell_line_bulk, f'adata_{cell_line_bulk}.h5ad'))

        # - read adata_rna
        save_dir = f'/nfs/public/lichen/data/single_cell/cell_line/CNP0003658/process/RNA/{cell_line_single}'
        adata_rna_raw = sc.read(os.path.join(save_dir, f'adata_rna_{cell_line_single}.h5ad'))
    
    if not isinstance(adata_rna.X, np.ndarray):
        adata_rna.X = adata_rna.X.toarray()
    
    # - get common pert
    adata_L1000_sub = adata_L1000[adata_L1000.obs['cell_id']==cell_line_bulk]
    L1000_total_perts = np.unique(adata_L1000_sub.obs['pert_iname'])
    common_perts = L1000_total_perts

    print('common_perts num: ', len(common_perts))
    
    # - get common var
    common_var = np.intersect1d(adata_rna.var_names, adata_L1000.var_names)
    
    adata_pert_list = []
    pert_gene_rank_dict = {}

    for pert in tqdm(common_perts):
        
        sub_idx = (adata_L1000.obs['cell_id']==cell_line_bulk) & (adata_L1000.obs['pert_iname']==pert)
        gene_score_L1000 = pd.DataFrame({'gene': list(adata_L1000.var_names), 'z-score-L1000': adata_L1000[sub_idx].X[0, :]})
        gene_score_L1000 = gene_score_L1000.sort_values(by='z-score-L1000', ascending=False)
        
        gene_score_L1000 = gene_score_L1000[gene_score_L1000['gene'].isin(common_var)]
        
        gene_score_merge = pd.concat([gene_score_L1000.head(30), gene_score_L1000.tail(30)])

        if save_prefix == 'mean':
            mean_value = np.mean(np.array(adata_rna[:, list(gene_score_merge['gene'])].X).ravel())
        if save_prefix == 'mean_v2':
            mean_value = np.mean(np.array(adata_rna[:, list(gene_score_L1000['gene'])].X).ravel())
        
        # adata_pert = adata_rna.copy()
        # adata_pert.X.fill(mean_value)
        
        # adata_pert.obs_names = [i+f'_{pert}' for i in adata_pert.obs_names]
        
        # # - adata_ctrl
        # adata_ctrl = adata_rna.copy()
        
        # adata_ctrl.obs['batch'] = 'ctrl'
        # adata_pert.obs['batch'] = 'pert'
        
        # adata_concat = ad.concat([adata_ctrl, adata_pert])
        
        # # - cal de genes
        # rankby_abs = False

        # sc.tl.rank_genes_groups(
        #     adata_concat,
        #     groupby='batch',
        #     reference='ctrl',
        #     rankby_abs=rankby_abs,
        #     n_genes=len(adata_concat.var),
        #     use_raw=False,
        #     method = 'wilcoxon'
        # )
        # de_genes = pd.DataFrame(adata_concat.uns['rank_genes_groups']['names'])
        # pvals = pd.DataFrame(adata_concat.uns['rank_genes_groups']['pvals'])
        # pvals_adj = pd.DataFrame(adata_concat.uns['rank_genes_groups']['pvals_adj'])
        # scores = pd.DataFrame(adata_concat.uns['rank_genes_groups']['scores'])
        # logfoldchanges = pd.DataFrame(adata_concat.uns['rank_genes_groups']['logfoldchanges'])


        z_score_list = [1 if mean_value - value > 0 else -1 for value in np.mean(adata_rna.X, axis = 0)]
        # - get gene_score
        gene_score = pd.DataFrame({'gene':list(adata_rna.var_names),
                                   'z-score':list(z_score_list)})
        
        pert_gene_rank_dict[pert] = (list(gene_score['gene']), list(gene_score['z-score']))
        
        

        # break
        
    save_dir = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark_202410/zero_shot/result'
    save_prefix = f'{save_prefix_2}/{cell_line_bulk}' # use result of K562 to do the direct transfer
    os.makedirs(os.path.join(save_dir, save_prefix), exist_ok=True)

    import json
    with open(os.path.join(save_dir, save_prefix, 'pert_gene_rank_dict.json'), 'w') as f:
        json.dump(pert_gene_rank_dict, f)
        
        
    # break

==================== cell line is A549


/data1/lichen/anaconda3/envs/scGPT/lib/python3.9/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/data1/lichen/anaconda3/envs/scGPT/lib/python3.9/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


common_perts num:  3620


100%|██████████| 3620/3620 [00:49<00:00, 73.56it/s]


==================== cell line is HepG2


...

100%|██████████| 3780/3780 [00:51<00:00, 72.82it/s]


==================== cell line is A375


/data1/lichen/anaconda3/envs/scGPT/lib/python3.9/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/data1/lichen/anaconda3/envs/scGPT/lib/python3.9/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


common_perts num:  3669


100%|██████████| 3669/3669 [00:51<00:00, 71.67it/s]


In [26]:
os.path.join(save_dir, save_prefix)

'/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark_202410/zero_shot/result/mean_v2/A549/HEPG2/HT29/MCF7/SW480/PC3/A375'

In [15]:
mean_value = np.mean(np.array(adata_rna[:, list(gene_score_merge['gene'])].X).ravel())

ValueError: setting an array element with a sequence.

In [16]:
adata_rna[:, list(gene_score_merge['gene'])].X

<169x60 sparse matrix of type '<class 'numpy.float32'>'
	with 3678 stored elements in Compressed Sparse Row format>

In [13]:
np.mean(adata_rna.X, axis = 0).shape

(5155,)

In [5]:
mean_value

0.63962

In [8]:
adata_pert = adata_rna.copy()
adata_pert.X.fill(mean_value)

In [9]:
adata_pert.X

array([[0.63962, 0.63962, 0.63962, ..., 0.63962, 0.63962, 0.63962],
       [0.63962, 0.63962, 0.63962, ..., 0.63962, 0.63962, 0.63962],
       [0.63962, 0.63962, 0.63962, ..., 0.63962, 0.63962, 0.63962],
       ...,
       [0.63962, 0.63962, 0.63962, ..., 0.63962, 0.63962, 0.63962],
       [0.63962, 0.63962, 0.63962, ..., 0.63962, 0.63962, 0.63962],
       [0.63962, 0.63962, 0.63962, ..., 0.63962, 0.63962, 0.63962]],
      dtype=float32)

# 并行化-在L1000上跑corrlation method并存储结果

In [ ]:
import os
import numpy as np
import pandas as pd
import scanpy as sc
from tqdm import tqdm
from sklearn.metrics import precision_recall_curve, auc
from scipy.spatial.distance import cdist
import concurrent.futures
import json

# 定义处理每个 cell_line_single 的函数
def process_cell_line(cell_line_bulk, cell_line_single, common_cell_line, adata_L1000):
    print('=' * 20, f'cell line is {cell_line_single}')

    #===================prepare data
    if cell_line_bulk in ['PC3', 'A375']:
        save_dir_adata = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/benchmark_data/L1000/single_cell_data/SCP542'
        adata_rna = sc.read(os.path.join(save_dir_adata, cell_line_bulk, f'adata_{cell_line_bulk}.h5ad'))
        
        # - read adata_rna_raw
        save_dir = f'/nfs/public/lichen/data/single_cell/cell_line/SCP542/process/{cell_line_bulk}'
        adata_rna_raw = sc.read(os.path.join(save_dir, f'adata.h5ad'))
    
    else:
        save_dir_adata = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/benchmark_data/L1000/single_cell_data/CNP0003658'
        adata_rna = sc.read(os.path.join(save_dir_adata, cell_line_bulk, f'adata_{cell_line_bulk}.h5ad'))

        # - read adata_rna
        save_dir = f'/nfs/public/lichen/data/single_cell/cell_line/CNP0003658/process/RNA/{cell_line_single}'
        adata_rna_raw = sc.read(os.path.join(save_dir, f'adata_rna_{cell_line_single}.h5ad'))

    # - consctrut corr mtx
    if not isinstance(adata_rna.X, np.ndarray):
        adata_rna.X = adata_rna.X.toarray()
    corr_mtx = np.corrcoef(adata_rna.X.T)
    
    # - get var_names
    var_names = list(adata_rna.var_names)
    
    # - get common pert
    adata_L1000_sub = adata_L1000[adata_L1000.obs['cell_id']==cell_line_bulk]
    L1000_total_perts = np.unique(adata_L1000_sub.obs['pert_iname'])
    common_perts = np.intersect1d(adata_rna.var_names, L1000_total_perts)

    
    
    print('common_perts num: ', len(common_perts))
    print('common var to L1000 data is: ', len(np.intersect1d(var_names, adata_L1000.var_names)))

    
    adata_pert_list = []
    pert_gene_rank_dict = {}
    count = 0
    for pert in tqdm(common_perts):
        count += 1
        # - get the corr change
        pert_value = adata_rna.X[:, var_names.index(pert)] # value of the pert gene, 500 cells
        pert_corr = corr_mtx[var_names.index(pert), :] # corr of the pert gene to total genes
        exp_change = np.dot(pert_value.reshape(-1, 1), pert_corr.reshape(1, -1))
        
        # - create adata_pert
        adata_pert = adata_rna.copy()
        adata_pert.X -= exp_change
        adata_pert.X[adata_pert.X < 0] = 0
        adata_pert.obs_names = [i+f'_{pert}' for i in adata_pert.obs_names]
        
        # - adata_ctrl
        adata_ctrl = adata_rna.copy()
        
        adata_ctrl.obs['batch'] = 'ctrl'
        adata_pert.obs['batch'] = 'pert'
        
        adata_concat = ad.concat([adata_ctrl, adata_pert])
        
        # - cal de genes
        rankby_abs = False

        sc.tl.rank_genes_groups(
            adata_concat,
            groupby='batch',
            reference='ctrl',
            rankby_abs=rankby_abs,
            n_genes=len(adata_concat.var),
            use_raw=False,
            method = 'wilcoxon'
        )
        de_genes = pd.DataFrame(adata_concat.uns['rank_genes_groups']['names'])
        pvals = pd.DataFrame(adata_concat.uns['rank_genes_groups']['pvals'])
        pvals_adj = pd.DataFrame(adata_concat.uns['rank_genes_groups']['pvals_adj'])
        scores = pd.DataFrame(adata_concat.uns['rank_genes_groups']['scores'])
        logfoldchanges = pd.DataFrame(adata_concat.uns['rank_genes_groups']['logfoldchanges'])

        # - get gene_score
        gene_score = pd.DataFrame({'gene':list(de_genes['pert']),
                                   'z-score':list(scores['pert'])})
        
        pert_gene_rank_dict[pert] = (list(de_genes['pert']), list(scores['pert']))
        
        
    save_dir = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark_202410/zero_shot/result'
    save_prefix = f'correlation_method/{cell_line_bulk}' # use result of K562 to do the direct transfer
    os.makedirs(os.path.join(save_dir, save_prefix), exist_ok=True)

    import json
    with open(os.path.join(save_dir, save_prefix, 'pert_gene_rank_dict.json'), 'w') as f:
        json.dump(pert_gene_rank_dict, f)
        
        
# 主函数
if __name__ == "__main__":
    # - get cell line name
    common_cell_line = \
    {   'A549': 'A549',
        'HEPG2': 'HepG2',
        'HT29': 'HT29',
        'MCF7': 'MCF7',
        # 'SKBR3': 'SK-BR-3',
        'SW480': 'SW480',
        'PC3': 'PC3',
        'A375': 'A375',
    } # L1000 cell line : single-cell cell line

    # - read adata_L1000, this is processed data
    adata_L1000 = sc.read('/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/benchmark_data/L1000/GSE92742/adata_gene_pert.h5ad')


    # 使用并行执行
    with concurrent.futures.ProcessPoolExecutor() as executor:
        futures = [
            executor.submit(process_cell_line, cell_line_bulk, common_cell_line[cell_line_bulk], common_cell_line, adata_L1000)
            for cell_line_bulk in common_cell_line.keys()
        ]
        
        # 等待所有任务完成
        for future in concurrent.futures.as_completed(futures):
            try:
                future.result()  # 获取每个任务的结果，如果有异常，将在此处抛出
            except Exception as e:
                print(f"Error: {e}")


/data1/lichen/anaconda3/envs/scGPT/lib/python3.9/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


==================== cell line is A549


/data1/lichen/anaconda3/envs/scGPT/lib/python3.9/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/data1/lichen/anaconda3/envs/scGPT/lib/python3.9/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


==================== cell line is HepG2


/data1/lichen/anaconda3/envs/scGPT/lib/python3.9/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/data1/lichen/anaconda3/envs/scGPT/lib/python3.9/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


...

  0%|          | 5/3091 [00:01<16:01,  3.21it/s]t]

common_perts num:  3086
common var to L1000 data is:  930


  0%|          | 7/3086 [00:05<39:37,  1.29it/s]] 

common_perts num:  3107
common var to L1000 data is:  945


100%|██████████| 3096/3096 [46:26<00:00,  1.11it/s]
